In [1]:
# to handle dataset
import pandas as pd
import datetime as dt
import numpy as np

#to visualize
import matplotlib.pyplot as plt
%matplotlib inline

#ols regression 
import statsmodels.formula.api as smf

# feature scaling
from sklearn.preprocessing import StandardScaler

#to crawl stock data
import pandas_datareader.data as web

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
start = dt.datetime(2018,1,1)
end = dt.datetime(2020,1,1)
# end = dt.datetime.now() 

asii = web.DataReader("ASII.JK", 'yahoo', start,end)
dji = web.DataReader("^DJI",'yahoo',start,end)
nikkei = web.DataReader("^N225",'yahoo', start, end)
sp500 = web.DataReader("^GSPC",'yahoo', start, end)
hsi = web.DataReader("^HSI", 'yahoo',start, end)
ihsg = web.DataReader("^JKSE", 'yahoo', start, end)


## Step 1: Data Munging

In [4]:
#bind all stock market data into same dataframe
# df is the dataframe of our tradng model
# due to the timezone issues, we extract and calculate approriate stock market
#  data for analysis 
df = pd.DataFrame(index=asii.index)
#df['asii'] = np.log(asii['Open'].shift(-1)) - np.log(asii['Open'])
df['asii'] = asii['Open'].shift(-1) - asii['Open']
df['asii_lag1'] = df['asii'].shift(1)   
df['dji'] = dji['Open'] - dji['Open'].shift(1)
df['sp500'] = sp500['Open']- sp500['Open'].shift(1)
df['hsi'] = hsi['Open']- hsi['Close']
df['nikkei'] = nikkei['Open'] - nikkei['Close']
df['ihsg'] = ihsg['Open'] - ihsg['Open'].shift(1)
df['Price'] = asii['Open']
df.tail()

,asii,asii_lag1,dji,sp500,hsi,nikkei,ihsg,Price
Date,,,,,,,,
2019-12-20,150.0,-25.0,330.330078,31.010010,-36.798828,76.818359,-17.228027,6850.0
2019-12-23,-200.0,150.0,-116.861328,2.719971,118.208984,100.179688,52.510742,7000.0
2019-12-26,125.0,-200.0,-33.109375,1.750000,NaN,-137.220703,-6.611816,6800.0
2019-12-27,0.0,125.0,135.878906,20.030029,-191.759766,116.029297,18.509766,6925.0
2019-12-30,NaN,0.0,-20.580078,-7.139893,-68.910156,114.310547,7.565918,6925.0


In [5]:
#checking for NaN Values in dataframe
df.isnull().mean()

asii         0.001927
asii_lag1    0.001927
dji          0.036609
sp500        0.036609
hsi          0.055877
nikkei       0.065511
ihsg         0.067437
Price        0.000000
dtype: float64

In [6]:
# use fillna() from dataframe to forwad filling the NaN values
# then we can drop the reminding Nan values
df = df.fillna(method='ffill')
df = df.dropna()

print(df.shape)

(516, 8)


## Step 2 : Split Data

In [7]:
# Split the data into train set and test set (out of sample)
train = df.iloc[0:300, :] 
test = df.iloc[300:516, :]

In [8]:
print(train.shape, test.shape)
train.head()

(300, 8) (216, 8)


,asii,asii_lag1,dji,sp500,hsi,nikkei,ihsg,Price
Date,,,,,,,,
2018-01-04,75.0,-175.0,114.410156,21.459961,-44.769531,-432.599609,-83.554199,8075.0
2018-01-05,150.0,75.0,150.060547,12.020020,79.218750,-71.529297,21.740234,8150.0
2018-01-08,0.0,150.0,193.480469,11.339844,-4.439453,-71.529297,74.904785,8300.0
2018-01-09,-25.0,0.0,3.650391,8.479980,-77.900391,98.980469,39.512207,8300.0
2018-01-10,0.0,-25.0,36.080078,-5.599854,-64.480469,44.611328,-12.956055,8275.0


## Step 3: Explore the train data set

In [ ]:
# generate scatter matrix among all stock markets to observe the association

from pandas.plotting import scatter_matrix
sm = scatter_matrix(train, figsize=(10, 10))

In [ ]:
corr_array = train.iloc[:, :].corr()['asii']
print(corr_array)

In [ ]:
formula = 'asii ~ asii_lag1 + dji + sp500 + hsi + nikkei + ihsg + Price'
lm = smf.ols(formula= formula, data=train).fit()
lm.summary()

## Step 4: Make Prediction

In [ ]:
train['prediction'] = lm.predict(train)
test['prediction'] = lm.predict(test)
plt.scatter(train['asii'], train['prediction'])

## Step 5: Model Evaluation - Statistical Standard


In [ ]:
# RMSE - Root Mean Squared Error, Adjusted R^2
def adjustedMetric(data, model, model_k, yname):
    data['yhat'] = model.predict(data)
    SST = ((data[yname] - data[yname].mean())**2).sum()
    SSR = ((data['yhat'] - data[yname].mean())**2).sum()
    SSE = ((data[yname] - data['yhat'])**2).sum()
    r2 = SSR/SST
    adjustR2 = 1 - (1-r2)*(data.shape[0] - 1)/(data.shape[0] -model_k -1)
    RMSE = (SSE/(data.shape[0] -model_k -1))**0.5
    return adjustR2, RMSE

In [ ]:
def assessTable(test, train, model, model_k, yname):
    r2test, RMSEtest = adjustedMetric(test, model, model_k, yname)
    r2train, RMSEtrain = adjustedMetric(train, model, model_k, yname)
    assessment = pd.DataFrame(index=['R2', 'RMSE'], columns=['Train', 'Test'])
    assessment['Train'] = [r2train, RMSEtrain]
    assessment['Test'] = [r2test, RMSEtest]
    return assessment

In [ ]:
# get the assesment table for model
assessTable(test,train,lm,7,'asii')

## Step 6: Profit of Signal-based strategy

In [ ]:
# Train
train['Order'] = [1 if sig >0 else -1 for sig in train['prediction']]
train['Profit'] = train['asii'] * train['Order']

train['Wealth'] =train['Profit'].cumsum()
print('Total profit made in train ', train['Profit'].sum())

In [ ]:
train.head()

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Performance of Strategy in Train')
plt.plot(train['Wealth'].values, color='green', label='Signal based strategy')
plt.plot(train['asii'].cumsum().values, color='red', label='Buy and Hold strategy')
plt.legend()
plt.show()

In [ ]:
# Test
test['Order'] = [1 if sig >0 else  -1 for sig in test['prediction']]
test['Profit'] = test['asii'] * test['Order']

test['Wealth'] = test['Profit'].cumsum()
print ('Total profit made in Test: ', test['Profit'].sum())

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Performance of Strategy in Test')
plt.plot(test['Wealth'].values, color='green', label='Signal based strategy')
plt.plot(test['asii'].cumsum().values, color='red', label='Buy and Hold strategy')
plt.legend()
plt.show()

## Final Step: Evaluation of model - Practical Standard
There are 2 common practical standard to evaluate trading model performance. _Sharpe Ratio_ and _Maximum Drawdown_

In [ ]:
train['Wealth'] = train['Wealth'] + train.loc[train.index[0],'Price']
test['Wealth'] = test['Wealth'] + test.loc[test.index[0],'Price']

In [ ]:
# Sharpe Ratio on Train data
train['Return'] = np.log(train['Wealth']) - np.log(train['Wealth'].shift(1))
dailyr = train['Return'].dropna()

print('Daily Sharpe Ratio is ', dailyr.mean()/dailyr.std(ddof=1))
print('Yearly Sharpe Ratio is ', (252**0.5)*dailyr.mean()/dailyr.std(ddof=1))

In [ ]:
# Sharpe Ratio on Test data
test['Return'] = np.log(test['Wealth']) - np.log(test['Wealth'].shift(1))
dailyr = test['Return'].dropna()

print('Daily Sharpe Ratio is ', dailyr.mean()/dailyr.std(ddof=1))
print('Yearly Sharpe Ratio is ', (252**0.5)*dailyr.mean()/dailyr.std(ddof=1))

In [ ]:
# Maximum Drawdown in train data
train['Peak'] = train['Wealth'].cummax()
train['Drawdown'] = (train['Peak']- train['Wealth'])/train['Peak']
print('Maximum Drawdown in train is ', train['Drawdown'].max())

In [ ]:
# Maximum Drawdown in test data
test['Peak'] = test['Wealth'].cummax()
test['Drawdown'] = (test['Peak']- test['Wealth'])/test['Peak']
print('Maximum Drawdown in train is ', test['Drawdown'].max())